# Bert NER on SageMaker using PyTorch

This uses the Biocreative II gene mention dataset https://biocreative.bioinformatics.udel.edu/tasks/biocreative-ii/task-1a-gene-mention-tagging/






In [1]:
import sys, os
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [2]:
import sagemaker, boto3

from sagemaker import get_execution_role
sm_session = sagemaker.session.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')

# role=get_execution_role()
role ="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20181222T162635".format(account_id)


2020-10-04 19:51:52,617 - sagemaker.analytics - WARNING - pandas failed to import. Analytics features will be impaired or broken.
2020-10-04 19:51:53,090 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2020-10-04 19:51:53,256 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
data_bucket = "aegovan-data"

data_bucket_prefix = "Biocreative-gene-mention"

s3_uri_data = "s3://{}/{}/train data".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train.in")
s3_uri_classes = "{}/{}".format(s3_uri_data, "GENE.eval")

s3_uri_test = "s3://{}/{}/test data".format(data_bucket, data_bucket_prefix , "test.in")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint".format(data_bucket, data_bucket_prefix)

## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [4]:
inputs_full =  {
    "train" : s3_uri_train,
    "class" : s3_uri_classes
}

inputs = inputs_full

In [5]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [6]:
instance_type = "ml.p3.2xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [17]:
hp = {
"epochs" : 30,
"earlystoppingpatience" : 5,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"classfile":s3_uri_classes.split("/")[-1],
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2,
"log-level" : "INFO"
}



In [18]:
hp

{'epochs': 30,
 'earlystoppingpatience': 5,
 'batch': 8,
 'trainfile': 'train.in',
 'classfile': 'GENE.eval',
 'gradaccumulation': 4,
 'log-level': 'INFO',
 'maxseqlen': 512,
 'lr': 1e-05,
 'finetune': 0,
 'checkpointdir': '/opt/ml/checkpoints/',
 'checkpointfreq': 2}

In [19]:
inputs

{'train': 's3://aegovan-data/Biocreative-gene-mention/train data/train.in',
 'class': 's3://aegovan-data/Biocreative-gene-mention/train data/GENE.eval'}

In [20]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [21]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample


In [22]:
job_type = "bc2-ner-bert"
base_name = "{}".format(job_type)

In [51]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir
                    )

estimator.fit(inputs, wait=True)

2020-10-05 01:09:32,999 - sagemaker - INFO - Creating training-job with name: bc2-ner-bert-2020-10-04-14-09-30-400
2020-10-04 14:09:34 Starting - Starting the training job...

KeyboardInterrupt: 

## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [57]:
import sagemaker
training_job = "bc2-ner-bert-2020-10-04-14-09-30-400"
estimator = sagemaker.estimator.Estimator.attach(training_job)


2020-10-04 16:06:52 Starting - Preparing the instances for training
2020-10-04 16:06:52 Downloading - Downloading input data
2020-10-04 16:06:52 Training - Training image download completed. Training in progress.
2020-10-04 16:06:52 Uploading - Uploading generated training model
2020-10-04 16:06:52 Completed - Training job completed


In [58]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
role = role

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     framework_version='1.4.0',
                     py_version = "py3",
                     entry_point='serve.py',
                     source_dir='src'
                    
                    )

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

2020-10-05 09:40:30,033 - sagemaker - INFO - Creating model with name: pytorch-inference-2020-10-04-22-40-30-033
2020-10-05 09:40:33,921 - sagemaker - INFO - Creating endpoint with name pytorch-inference-2020-10-04-22-40-31-479
----------------!

### Invoke API

In [59]:
class Predictor:
    
    def serialize(self, x):
        return x
    
    def deserialize(self,x, content_type):
        payload_bytes = json.loads( x.read().decode("utf-8") )
        return payload_bytes

In [60]:
#predictor= sagemaker.predictor.Predictor("end")
predictor.serializer = Predictor()
predictor.deserializer = Predictor()


In [136]:
import json

from datasets.biocreative_dataset import BiocreativeDataset
from datasets.biocreative_ner_label_mapper import BiocreativeNerLabelMapper

def chunk(l, size=5):
    for i in range(0, len(l),size):
        yield l[i:i+size]

with open("tmp/test.in", "r") as f:
    d = []
    ids =[]
    for l in f.readlines():
        l = l.rstrip("\n")
        (id, text) = l.split(" ")[0], " ".join(l.split(" ")[1:])
        d.append(text)
        ids.append(id)
        
    
    
label_mapper = BiocreativeNerLabelMapper()

id_chucks = list(chunk(ids))
result = []
for (i, data) in enumerate(chunk(d)):

    data_bytes=("\n".join(data)).encode("utf-8")
    response  = predictor.predict(data_bytes,  
                                    initial_args={ "Accept":"text/json", "ContentType" : "text/csv" }
                                       )

    assert len(response) == len(data), "Data size {} doesnt match result size {}".format(len(r), len(d))



    for ri, r in enumerate(response):
        doc_id = id_chucks[i][ri]

        result.append({"docid":doc_id, "text": data[ri], "entities_detected": r })
        
    
    
with open("result.json", "w") as f:
    json.dump( result, f)




## Delete endpoint

In [55]:
predictor.delete_endpoint()

2020-10-05 01:10:58,764 - sagemaker - INFO - Deleting endpoint configuration with name: pytorch-inference-2020-10-04-03-34-30-786
2020-10-05 01:11:00,697 - sagemaker - INFO - Deleting endpoint with name: end
